## データ調査
- 予備実験2後のデータを確認して、データの要素を確認する
    - どのデータなら有効的であると考えられるか, あるいはどのデータは効果ないのかを明確にすることが目的

In [1]:
import glob
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data
from sklearn.metrics import classification_report

import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib

In [2]:
# data/result/に格納されている全csvファイルの読み込み
csv_files = glob.glob(os.path.join("../../data/result", "*.csv"))
df_list = []
for file in csv_files:
    tmp_df = pd.read_csv(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

df.shape

(56796, 3)

In [3]:
# predの要素で, propertyが含まれる値を返す
df_prop = df[df.pred.str.contains('property')]

# 'wikiPage'を含まないpropertyを返す
df_prop = df_prop[~df_prop['pred'].str.contains('wikiPage')]

# '画像'を含まないpropertyを返す
df_prop = df_prop[~df_prop['pred'].str.contains('画像')]

# predの頻出上位10の要素のindex_listを取得
top_10_df_prop = df_prop.groupby('pred').count().sort_values(['obj'], ascending=False).head(10)
top_10_prop_list = top_10_df_prop.index.to_list()

# 頻出上位10を持つdataの抽出
top_10_df = df[df['pred'].isin(top_10_prop_list)]
display(top_10_df.shape)

# predを目的変数(label)として扱うための準備
prop_to_label = dict(zip(top_10_prop_list, range(0,10)))
column_name = dict(map(lambda k: (k[1],k[0].split("/")[-1]), prop_to_label.items()))

(1049, 3)

In [4]:
# モデル読み込み
model = Word2Vec.load('../../Models/japanese-word2vec-model-builder/word2vec.gensim.model')

In [5]:
# モデルを利用した文字のエンべディング
## 未知語に関しては, 今回はNaNで対応
def vectorize(model, word):
    try:
        output = model.wv[word]
        return output
    except:
        return np.nan

In [6]:
# obj要素に関してはLOD同士が繋がっているため, 基本的にURIで記述されている.
# また, (県の魚:〇〇)の要素や数値データが入っている場合もあるため, それぞれ最後尾の要素を値として扱う. 
def preprocessing(obj):
    if type(obj) != str:
        output = obj
    elif len(obj.split("：")) != 1:
        output = obj.split("：")[-1]
    else:
        output = obj.split("/")[-1]
    return output

In [7]:
# vectorization
## key要素(本データでは〇〇県)は,単純エンベディング対象.
## obj要素は, 上記の関数で整えた後に, エンベディング.

cp_top_10_df = top_10_df.copy()
cp_top_10_df['key_vec'] = cp_top_10_df['key'].map(lambda x:vectorize(model,x))
cp_top_10_df['label'] = cp_top_10_df['pred'].map(prop_to_label)
obj_list = cp_top_10_df['obj'].map(lambda x:preprocessing(x))
cp_top_10_df['obj_vec'] = obj_list.map(lambda x:vectorize(model,x))

In [8]:
# ベクトル化前のデータとベクトル化後のデータの結合 + 未知語のNaNの排除
top_10_df_vec = pd.merge(top_10_df,cp_top_10_df, left_index=True, right_on=top_10_df.index)
top_10_df_vec = top_10_df_vec.reset_index(drop=True)
top_10_df_vec = top_10_df_vec.drop(["key_0","key_y","pred_y","obj_y"], axis=1)
top_10_df_vec = top_10_df_vec.dropna().reset_index(drop=True)

top_10_df_vec.shape

(591, 6)

In [9]:
# 目的変数のバイナライズ + 説明変数の切り出し
lb = LabelBinarizer()
lb.fit(top_10_df_vec.label)
label_dence = lb.fit_transform(top_10_df_vec.label)
input_data = list(top_10_df_vec['obj_vec'].values)

In [10]:
# マルチラベル分類
forest = RandomForestClassifier(n_estimators = 500, random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(input_data, label_dence)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=500,
                                                       random_state=1),
                      n_jobs=-1)

In [11]:
estimator = multi_target_forest.estimators_
y_pred = multi_target_forest.predict(input_data)
y_pred_proba = multi_target_forest.predict_proba(input_data)

In [12]:
list(column_name.values())

['隣接都道府県',
 'before',
 '歌など',
 '所在地',
 '説明',
 '表記',
 'years',
 'after',
 'シンボル名',
 'title']

In [13]:
np_pred = np.array(y_pred)
np_proba = np.array(y_pred_proba)
np_proba_trans = np_proba.transpose(1,0,2)

In [14]:
# ラベル有の閾値の設定
pred_2 = (np_proba_trans[:,:,1] > 0.2).astype(int)

In [15]:
df = pd.DataFrame(np_pred)
df_2 = pd.DataFrame(pred_2)
df_2_new = df_2.rename(columns=column_name)

In [16]:
df_pre_obj = top_10_df_vec.loc[:,['pred_x','obj_x','obj_vec']]
df_2_pre_obj = pd.concat([df_pre_obj,df_2_new],axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_2_pre_obj.loc[:,:'obj_vec'], df_2_pre_obj.loc[:,'隣接都道府県':], random_state=1)

In [18]:
print(len(X_train))
print(len(y_train))

443
443


In [19]:
print(X_train.obj_vec.values.shape)
print(y_train.values.shape)

(443,)
(443, 10)


In [20]:
# マルチラベル分類
multi_target_forest.fit(X_train.obj_vec.values.tolist(), y_train.values)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=500,
                                                       random_state=1),
                      n_jobs=-1)

In [21]:
estimator = multi_target_forest.estimators_
y_pred = multi_target_forest.predict(X_test.obj_vec.values.tolist())
y_pred_proba = multi_target_forest.predict_proba(X_test.obj_vec.values.tolist())

In [22]:
multi_pred_df = pd.DataFrame(y_pred)
base2_output = multi_pred_df.rename(columns=column_name).add_prefix('output_')
base2_output.head()

,output_隣接都道府県,output_before,output_歌など,output_所在地,output_説明,output_表記,output_years,output_after,output_シンボル名,output_title
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0


In [23]:
test_data = X_test.loc[:,:'obj_x'].reset_index(drop=True)
test_data.head()

,pred_x,obj_x
0,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/黒川藩
1,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/越前国
2,http://ja.dbpedia.org/property/years,1880
3,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/名東県
4,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県


In [24]:
base1_output = y_test.reset_index(drop=True)
base1_output.head()

,隣接都道府県,before,歌など,所在地,説明,表記,years,after,シンボル名,title
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,1,1,0,0,0,0,0,0,0,0


In [25]:
print("\n--------予備実験1の出力結果----------")
base1 = pd.concat([test_data,base1_output],axis=1)
display(base1.shape)
display(base1.head(5))

print("\n--------予備実験2の出力結果----------")
base2 = pd.concat([test_data,base2_output],axis=1)
display(base2.shape)
display(base2.head(5))


--------予備実験1の出力結果----------


(148, 12)

,pred_x,obj_x,隣接都道府県,before,歌など,所在地,説明,表記,years,after,シンボル名,title
0,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/黒川藩,0,1,0,0,0,0,0,0,0,0
1,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/越前国,0,1,0,0,0,0,0,0,0,0
2,http://ja.dbpedia.org/property/years,1880,0,0,0,0,0,0,1,0,0,0
3,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/名東県,0,1,0,0,0,0,0,1,0,0
4,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県,1,1,0,0,0,0,0,0,0,0



--------予備実験2の出力結果----------


(148, 12)

,pred_x,obj_x,output_隣接都道府県,output_before,output_歌など,output_所在地,output_説明,output_表記,output_years,output_after,output_シンボル名,output_title
0,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/黒川藩,0,1,0,0,0,0,0,0,0,0
1,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/越前国,0,1,0,0,0,0,0,0,0,0
2,http://ja.dbpedia.org/property/years,1880,0,0,0,0,0,0,1,0,0,0
3,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/名東県,0,1,0,0,0,0,0,1,0,0
4,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県,1,1,0,0,0,0,0,0,0,0


## 調査内容
- 予備実験1の結果と予備実験2の結果を比較調査

In [26]:
# テストデータのpred要素のカウント
test_data.groupby('pred_x').count()

,obj_x
pred_x,
http://ja.dbpedia.org/property/after,3
http://ja.dbpedia.org/property/before,26
http://ja.dbpedia.org/property/title,1
http://ja.dbpedia.org/property/years,16
http://ja.dbpedia.org/property/所在地,15
http://ja.dbpedia.org/property/歌など,8
http://ja.dbpedia.org/property/表記,21
http://ja.dbpedia.org/property/説明,14
http://ja.dbpedia.org/property/隣接都道府県,44


In [59]:
# マルチラベルが付与されたテストデータ
base1_multi = base1[base1.loc[:,'隣接都道府県':].sum(axis=1) > 1]
base1_multi_index = base1_multi.index
display(base1_multi)

,pred_x,obj_x,隣接都道府県,before,歌など,所在地,説明,表記,years,after,シンボル名,title
3,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/名東県,0,1,0,0,0,0,0,1,0,0
4,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県,1,1,0,0,0,0,0,0,0,0
17,http://ja.dbpedia.org/property/歌など,県民の歌：県民の歌,0,0,1,0,0,0,0,0,1,0
29,http://ja.dbpedia.org/property/after,http://ja.dbpedia.org/resource/長崎県,1,0,0,0,0,0,0,1,0,0
32,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/長崎県,1,0,0,0,0,0,0,1,0,0
38,http://ja.dbpedia.org/property/title,http://ja.dbpedia.org/resource/日本の首都,0,0,0,0,1,0,0,0,0,1
46,http://ja.dbpedia.org/property/歌など,マスコット：くまモン,0,0,1,0,1,0,0,0,0,0
81,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/石川県,1,1,0,0,0,0,0,0,0,0
90,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/鹿児島県,1,0,0,0,0,0,0,1,0,0
92,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/高知県,1,1,0,0,0,0,0,0,0,0


In [60]:
base2_single = base2_base1_multi[base2_base1_multi.loc[:,'output_隣接都道府県':].sum(axis=1) < 2]
display(base2_single)

,pred_x,obj_x,output_隣接都道府県,output_before,output_歌など,output_所在地,output_説明,output_表記,output_years,output_after,output_シンボル名,output_title
38,http://ja.dbpedia.org/property/title,http://ja.dbpedia.org/resource/日本の首都,0,0,0,0,1,0,0,0,0,0
46,http://ja.dbpedia.org/property/歌など,マスコット：くまモン,0,0,0,0,1,0,0,0,0,0
104,http://ja.dbpedia.org/property/歌など,県民の歌：最上川,0,0,0,0,1,0,0,0,0,0


### **隣接都道府県 before after について**

In [67]:
display(base1_multi[(base1_multi.pred_x.str.contains('隣接都道府県') |
                     base1_multi.pred_x.str.contains('before') |
                     base1_multi.pred_x.str.contains('after')
                )])

,pred_x,obj_x,隣接都道府県,before,歌など,所在地,説明,表記,years,after,シンボル名,title
3,http://ja.dbpedia.org/property/before,http://ja.dbpedia.org/resource/名東県,0,1,0,0,0,0,0,1,0,0
4,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県,1,1,0,0,0,0,0,0,0,0
29,http://ja.dbpedia.org/property/after,http://ja.dbpedia.org/resource/長崎県,1,0,0,0,0,0,0,1,0,0
32,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/長崎県,1,0,0,0,0,0,0,1,0,0
81,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/石川県,1,1,0,0,0,0,0,0,0,0
90,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/鹿児島県,1,0,0,0,0,0,0,1,0,0
92,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/高知県,1,1,0,0,0,0,0,0,0,0
137,http://ja.dbpedia.org/property/隣接都道府県,http://ja.dbpedia.org/resource/栃木県,1,1,0,0,0,0,0,0,0,0


- マルチラベルが検出された隣接都道府県と行政区の設置場所(before, after)について
    - 抽出データ(頻出上位10)では、この3種のデータが互いに検出されることが多い.
        - 行政区の設置場所(before, after)は時系列であるが, 現在から見ると全て過去の事例である.
            - 現在以前の行政区の設置場所と捉え, まとめることは可能であると考えられる.
        - 隣接都道府県に関しては, 行政区の設置場所が隣接の場合とそうではない場合が存在する.
            - 関係同士をまとめるのではなく, (以前の行政区である)かつ(現在の隣接都道府県である)を満たすデータをまとめる条件とする必要がある.
    - 上記の結果から以下の分岐が考えられる.
        1. 関係リンク同士をまとめることができる.
            - 具体例: before, after
        2. データの一部だけをまとめることができる.
        3. データをまとめることができない.
            - 具体例: 隣接都道府県, (before, after)
            - リンク元データ(都道府県名データ)を用いて確認することは出来そう.

- keyデータを用いてobjデータを解明していく方法
    - keyデータとobjデータがあれば, 隣接都道府県であるかどうかを解明することが出来そう.

### **隣接都道府県 before after 以外について**

In [75]:
print("----------ベース実験1-------------")
display(base1_multi[(~base1_multi.pred_x.str.contains('隣接都道府県') &
                     ~base1_multi.pred_x.str.contains('before') &
                     ~base1_multi.pred_x.str.contains('after')
                )])

print('\n')
print("----------ベース実験2-------------")
display(base2_single)

----------ベース実験1-------------


,pred_x,obj_x,隣接都道府県,before,歌など,所在地,説明,表記,years,after,シンボル名,title
17,http://ja.dbpedia.org/property/歌など,県民の歌：県民の歌,0,0,1,0,0,0,0,0,1,0
38,http://ja.dbpedia.org/property/title,http://ja.dbpedia.org/resource/日本の首都,0,0,0,0,1,0,0,0,0,1
46,http://ja.dbpedia.org/property/歌など,マスコット：くまモン,0,0,1,0,1,0,0,0,0,0
104,http://ja.dbpedia.org/property/歌など,県民の歌：最上川,0,0,1,0,1,0,0,0,0,0




----------ベース実験2-------------


,pred_x,obj_x,output_隣接都道府県,output_before,output_歌など,output_所在地,output_説明,output_表記,output_years,output_after,output_シンボル名,output_title
38,http://ja.dbpedia.org/property/title,http://ja.dbpedia.org/resource/日本の首都,0,0,0,0,1,0,0,0,0,0
46,http://ja.dbpedia.org/property/歌など,マスコット：くまモン,0,0,0,0,1,0,0,0,0,0
104,http://ja.dbpedia.org/property/歌など,県民の歌：最上川,0,0,0,0,1,0,0,0,0,0


In [77]:
display(top_10_df[top_10_df.pred.str.contains('シンボル')].sample(3))
display(top_10_df[top_10_df.pred.str.contains('説明')].sample(3))

,key,pred,obj
56774,高知県,http://ja.dbpedia.org/property/シンボル名,他のシンボル
36869,岐阜県,http://ja.dbpedia.org/property/シンボル名,県の魚
5790,東京都,http://ja.dbpedia.org/property/シンボル名,都の歌


,key,pred,obj
31448,愛知県,http://ja.dbpedia.org/property/説明,名古屋臨海高速鉄道
18727,千葉県,http://ja.dbpedia.org/property/説明,県立九十九里自然公園（九十九里浜）
31454,愛知県,http://ja.dbpedia.org/property/説明,近畿日本鉄道


- 同じ関係リンクが結んでいる他のデータと比較しながら, まとめても問題ないかを判定する方法
    - 関係リンクの具体化とか関係リンクで見た時のブレなさを確認する.
    - keyデータではなく, objデータを主体とする考え.